In [1]:
import os 

#Import Tkinter. It is used to build graphical user interface using python module.
from tkinter import * 

In [2]:
# Read the resume sample from internal database
resume_path = r"C:\Users\yusuf\YusufSwoopTalent\TrainingDataset\named_entity\txt_resumes\15600_RESUME_Wells_resume_plain.txt"
training_data_dir_path = r"C:\Users\yusuf\YusufSwoopTalent\Keras practice\Yusuf Practice\Training_data"
file_name = '15600_RESUME_Wells_resume_plain.txt'
resume_raw_content = open(resume_path,"r", encoding='utf8').read()

In [3]:
# Define the two level labels as line_type and line_labels
line_types = {0:'content', 1:'title', 2:'header'}
line_labels = {0:'experience', 1:'skills', 2: 'education', 3:'personal', 4:'miscellaneous'}

In [ ]:
# Save the resume as list of sentences.
file_content = []
for line in resume_raw_content.split('\n'):
        line2 = line.strip()
        if line2 != '':
            file_content.append(line2)

In [3]:
#Create default labels for all the sentences.
def guess_line_type(line):
    return -1 

def guess_line_label(line):
    return -1

table_content = [[line, guess_line_type(line), guess_line_label(line)] for line in file_content]

In [4]:
# Class to create scrollbar for the gui
class ScrollableFrame(Frame):
    def __init__(self, parent, *args, **kw):
        '''
        Constructor
        '''
        Frame.__init__(self, parent, *args, **kw)
        # create a vertical scrollbar
        vscrollbar = Scrollbar(self, orient = VERTICAL)
        vscrollbar.pack(fill = Y, side = RIGHT, expand = TRUE)

        # create a horizontal scrollbar
        hscrollbar = Scrollbar(self, orient = HORIZONTAL)
        hscrollbar.pack(fill = X, side = BOTTOM, expand = TRUE)

        #Create a canvas object and associate the scrollbars with it
        self.canvas = Canvas(self, bd = 0, width=900, height=800, highlightthickness = 0, yscrollcommand = vscrollbar.set, xscrollcommand = hscrollbar.set)
        self.canvas.pack(side = LEFT, fill = BOTH, expand = TRUE)

        #Associate scrollbars with canvas view
        vscrollbar.config(command = self.canvas.yview)
        hscrollbar.config(command = self.canvas.xview)


        # set the view to 0,0 at initialization
        self.canvas.xview_moveto(0)
        self.canvas.yview_moveto(0)

        # create an interior frame to be created inside the canvas
        self.interior = interior = Frame(self.canvas)
        interior_id = self.canvas.create_window(0, 0, window=interior,
                anchor=NW)

        # track changes to the canvas and frame width and sync them,
        # also updating the scrollbar

        def _configure_interior(event):
            # update the scrollbars to match the size of the inner frame
            size = (interior.winfo_reqwidth(), interior.winfo_reqheight())
            self.canvas.config(scrollregion='0 0 %s %s' % size)
            if interior.winfo_reqwidth() != self.canvas.winfo_width():
                # update the canvas's width to fit the inner frame
                self.canvas.config(width = interior.winfo_reqwidth())
        interior.bind('<Configure>', _configure_interior)

In [5]:
class AnnotatorGui(Frame):
    def __init__(self, master, table_content):
        Frame.__init__(self, master=master)
#========================

        self.frame = ScrollableFrame(master)
        self.frame.pack(expand = TRUE)
#         self.label = Label(text="Shrink the window to activate the scrollbar.")
#         self.label.pack(expand = TRUE)
#=====================
        
        self.master.title("Annotate Resume Lines")

        self.master.rowconfigure(0, weight=1)
        self.master.columnconfigure(0, weight=1)
        self.frame.grid(sticky=W + E + N + S)
        self.frame.pack(expand = TRUE)

        self.line_index_label_list = []
        self.line_content_text_list = []
        self.line_type_button_list = []
        self.line_label_button_list = []

        for line_index, line in enumerate(table_content):
            self.build_line(table_content, line_index, line)

        self.rowconfigure(1, weight=1)
        self.columnconfigure(1, weight=1)

    def build_line(self, table_content, line_index, line):
        line_content = line[0]

        line_index_label = Label(self.frame.interior, width=5, height=1, text=str(line_index), bg='deep sky blue',fg='white')
        line_index_label.grid(row=line_index, column=0, sticky=W + E + N + S)
        line_content_text = Text(self.frame.interior, width=100, height=1.5, bg='sky blue')
        line_content_text.insert(INSERT, line_content)
        line_content_text.grid(row=line_index, column=1, sticky=W + E + N + S)
        
        def line_type_button_click(_line_index):
            line_type = table_content[_line_index][1]
            line_type = (line_type + 1) % len(line_types)
            table_content[_line_index][1] = line_type
            line_type_button["text"] = "Type: " + line_types[line_type]
            line_type_button.configure(bg="light blue")

        def line_label_button_click(_line_index):
            line_label = table_content[_line_index][2]
            line_label = (line_label + 1) % len(line_labels)
            table_content[_line_index][2] = line_label
            line_label_button["text"] = "Type: " + line_labels[line_label]
            line_label_button.configure(bg="light blue")

        line_type_button = Button(self.frame.interior, text="Type: Unknown", width=25,
                                  command=lambda: line_type_button_click(line_index))
        line_type_button.grid(row=line_index, column=2, sticky=W + E + N + S)
        line_label_button = Button(self.frame.interior, text='Label: Unknown', width=25,
                                   command=lambda: line_label_button_click(line_index))
        line_label_button.grid(row=line_index, column=3, sticky=W + E + N + S)

        if line[1] != -1:
            line_type_button["text"] = "Type: " + line_types[line[1]]
        if line[2] != -1:
            line_label_button["text"] = "Type: " + line_labels[line[2]]

In [6]:
#Main function to call to open tkinter GUI.
def gui_annotate(training_data_dir_path, index, file_path, file_content):

    root = Tk()

    table_content = [[line, guess_line_type(line), guess_line_label(line)] for line in file_content]
    gui = AnnotatorGui(root, table_content)

    def callback():
        root.destroy()
        output_file_path = os.path.join(training_data_dir_path, str(index) + '.txt')
        if os.path.exists(output_file_path):
            return
        with open(output_file_path, 'wt', encoding='utf8') as f:
            for line in table_content:
                line_content = line[0]
                data_type = line[1]
                label = line[2]

                if data_type == -1 or label == -1:
                    continue

                print('write line: ', line)
                f.write(line_types[data_type] + '\t' + line_labels[label] + '\t' + line_content)
                f.write('\n')

    root.protocol("WM_DELETE_WINDOW", callback)
    gui.mainloop()

In [8]:
gui_annotate(training_data_dir_path, file_name, 'file_path', file_content)

write line:  ['DAVID WELLS', 0, 3]
write line:  ['4779 Pewter Rd.', 0, 3]
write line:  ['Dwells52@yahoo.com', 0, 3]
write line:  ['Cincinnati, OH, 45244', 0, 3]
write line:  ['SENIOR COMMUNICATIONS DIRECTOR', 1, 0]
write line:  ['Strong combination of writing, research, analysis and crisis management skills', 0, 0]
write line:  ['Experience directing news coverage and editorial campaigns for a major metropolitan newspaper has given me the ability to quickly analyze and respond to any situation. I am able to craft and organize a public response and to develop long-term strategies for getting a message to targeted internal or external audiences. Core competencies include:', 0, 0]
write line:  ['Writing and Editing', 0, 0]
write line:  ['Cross-Functional Teams', 0, 0]
write line:  ['CONTRIBUTIONS & ACHIEVEMENTS', 2, 4]
write line:  ['Successfully directed news coverage for a major metropolitan newspaper, managing staffs from small teams to a news force of 70 reporters and a dozen editors.